# Imports and Config

In [1]:
import pandas as pd

# Load Data

In [2]:
raw = pd.read_csv("../data/biored_raw.csv")
raw

,instance_id,subreddit,post_id,sentence_id,post_title,text,annotated_labels,majority_labels,consensus_spans,num_annotators,span_level_agreement,group,auto_labels
0,104545801,offmychest,v502z3,0,Just ended my first relationship and I feel re...,I (22) ended my first relationship and feel gr...,"['[{""end"":6,""text"":""I (22)"",""start"":0,""labels""...",['per:age'],"[{'start': 0, 'end': 6, 'labels': ['per:age']}]",4,62.500000,group_0,age: information on age of a person;per: partn...
1,104545802,offmychest,yg091o,74,I had this idea of you,"And I have never known the love of my family, ...",[],['no_relation'],[],2,100.000000,group_0,per: parents: information on whether a person ...
2,104545803,offmychest,11zyznk,0,"I miss being a ""sex worker"" even though I'm in...","I , 23F , have been dating my boyfriend, 24M f...","['[{""end"":6,""text"":""I , 23"",""start"":0,""labels""...",['per:age'],"[{'start': 0, 'end': 6, 'labels': ['per:age']}]",5,50.238095,group_0,age: information on age of a person;sex or gen...
3,104545804,offmychest,xky3fc,1,everyone is losing love for me,The people in my life whom I owe to still bein...,"['[{""end"":79,""text"":""my mother and Girlfriend""...",['per: other family members'],"[{'start': 55, 'end': 79, 'labels': ['per: oth...",4,33.333333,group_0,per: children: information on whether a person...
4,104545805,offmychest,y0i42x,1,I absolutely HATE remote work,I live in a house with my boyfriend and two pe...,"['[{""end"":35,""text"":""my boyfriend"",""start"":23,...","['possessions', 'unclear / uncertain']","[{'start': 12, 'end': 17, 'labels': ['possessi...",3,2.777778,group_0,age: information on age of a person;per: child...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21475,108585405,Advice,ylp96d,9,Writing a resume as a new graduate?,Econometrics taught me valuable skills like ti...,"['[{""end"":39,""text"":""Econometrics taught me va...",['studies / education'],"[{'start': 0, 'end': 38, 'labels': ['studies /...",4,25.606713,group_5,studies / education: information on studies / ...
21476,108585407,Advice,xijdtc,4,Is it too late to change my career from scratc...,I'm 26 and will turn 27 in a few months... and...,"['[{""end"":90,""text"":""I\'m 26 and will turn 27 ...","['major life events', 'unclear / uncertain']",[],2,0.000000,group_5,age: information on age of a person;major life...
21477,108585408,Advice,11x514q,1,What should I do ?,"So backstory , im 19 and currently on my 2nd y...","['[{""end"":70,""text"":""im 19 and currently on my...",['studies / education'],"[{'start': 38, 'end': 70, 'labels': ['studies ...",4,67.831439,group_5,age: information on age of a person;studies / ...
21478,108585409,Advice,vnehh2,15,Im starting to hate my little sister's touch,I dont want her to grow up to resent me becaus...,"['[{""end"":76,""text"":""I refused to be a big sis...",['per: siblings'],"[{'start': 66, 'end': 76, 'labels': ['per: sib...",4,20.681063,group_5,per: siblings: information on whether a person...


...

In [1]:
import stanza


stanza.download('de')
nlp = stanza.Pipeline('de', processors='tokenize,pos,lemma,depparse')


text = "Angela Merkel war Präsidentin."

doc = nlp(text)

rows = []
for sent_id, sentence in enumerate(doc.sentences, start=1):
    for w in sentence.words:
        rows.append({
            "sentence_id": sent_id,
            "token_id": w.id,
            "text": w.text,
            "lemma": w.lemma,
            "upos": w.upos,
            "xpos": w.xpos,
            "feats": w.feats,
            "head": w.head,
            "deprel": w.deprel,
        })

df = pd.DataFrame(rows)
df

/Users/paulsilberbauer/PycharmProjects/NLP_Bioread_Reddit_2025W/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/paulsilberbauer/PycharmProjects/NLP_Bioread_Reddit_2025W/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-23 00:35:43 INFO: Downloaded file to /Users/paulsilberbauer/stanza_resources/resources.json
2025-10-23 00:35:43 INFO: Downloading default packages for language: de (German) ...
2025-10-23 00:35:45 INFO: File exists: /Users/paulsilberbauer/stanza_resources/de/default.zip
2025-10-23 00:35:49 INFO: Finished downloading models and saved to /Users/paulsilberbauer/stanza_resource

,sentence_id,token_id,text,lemma,upos,xpos,feats,head,deprel
0,1,1,Angela,Angela,PROPN,NE,Case=Nom|Gender=Fem|Number=Sing,4,nsubj
1,1,2,Merkel,Merkel,PROPN,NE,Case=Nom|Gender=Fem|Number=Sing,1,flat
2,1,3,war,sein,AUX,VAFIN,Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbF...,4,cop
3,1,4,Präsidentin,Präsidentin,NOUN,NN,Case=Nom|Gender=Fem|Number=Sing,0,root
4,1,5,.,.,PUNCT,$.,None,4,punct


In [13]:
# stanza.download("de")
nlp = stanza.Pipeline(
    "de",
    processors="tokenize,pos,lemma,depparse",
    tokenize_no_ssplit=True,
    use_gpu=False
)

def annotate_df(df: pd.DataFrame,
                text_col: str = "text",
                id_col: str = "instance_id",
                sent_col: str = "sentence_id") -> pd.DataFrame:
    rows = []
    for inst_val, text_val, sent_val in df[[id_col, text_col, sent_col]].itertuples(index=False, name=None):
        # skip invalid/empty text
        if not isinstance(text_val, str) or not text_val.strip():
            continue

        doc = nlp(text_val)
        if not doc.sentences:
            continue

        sent = doc.sentences[0]
        for w in sent.words:
            rows.append({
                "instance_id": inst_val,
                "sentence_id": sent_val,
                "token_id": w.id,
                "token": w.text,
                "lemma": w.lemma or "_",
                "upos": w.upos or "_",
                "xpos": w.xpos or "_",
                "feats": w.feats or "_",
                "head": w.head if w.head is not None else 0,
                "deprel": w.deprel or "_",
                "misc": w.misc if (w.misc and "SpaceAfter=No" in w.misc) else "_",
                "orig_text": text_val
            })

    return pd.DataFrame(rows)

df_first20 = raw.head(20).copy()
result_df = annotate_df(df_first20, text_col="text", id_col="instance_id", sent_col="sentence_id")
result_df.head()

2025-10-23 00:58:48 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-10-23 00:58:49 INFO: Downloaded file to /Users/paulsilberbauer/stanza_resources/resources.json
2025-10-23 00:58:49 WARNING: Language de package default expects mwt, which has been added
2025-10-23 00:58:50 INFO: Loading these models for language: de (German):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-10-23 00:58:50 INFO: Using device: cpu
2025-10-23 00:58:50 INFO: Loading: tokenize
2025-10-23 00:58:50 INFO: Loading: mwt
2025-10-23 00:58:50 INFO: Loading: pos
2025-10-23 00:58:52 INFO: Loading: lemma
2025-10-23 00:59:00 INFO: Loading: depparse
2025-10-23 00:59:01 INFO: Done 

,instance_id,sentence_id,token_id,token,lemma,upos,xpos,feats,head,deprel,misc,orig_text
0,104545801,0,1,I,I,PROPN,NE,_,0,root,_,I (22) ended my first relationship and feel gr...
1,104545801,0,2,(,(,PUNCT,$(,_,3,punct,_,I (22) ended my first relationship and feel gr...
2,104545801,0,3,22,22,NUM,CARD,NumType=Card,1,appos,_,I (22) ended my first relationship and feel gr...
3,104545801,0,4,),),PUNCT,$(,_,3,punct,_,I (22) ended my first relationship and feel gr...
4,104545801,0,5,ended,ended,X,FM,Foreign=Yes,1,appos,_,I (22) ended my first relationship and feel gr...
